In [1]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%autosave 100

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Autosaving every 100 seconds


In [0]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.utils.data import Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
quizes = np.empty((30, 81), np.float32)
solutions = np.empty((30, 81), np.int64)

i=0
f = open("/content/drive/My Drive/Colab Notebooks/sudoku_test.csv")
for line in f:
    quiz, solution = line.strip().split(",")
    quizes[i] = [int(i) for i in quiz]
    solutions[i] = [int(i) for i in solution]
    i += 1

quizes = quizes.reshape((-1, 9, 9))
solutions = solutions.reshape((-1, 9, 9))

print("quizes\n", quizes[0])
print("solutions\n", solutions[0])

quizes
 [[0. 8. 0. 0. 3. 2. 0. 0. 1.]
 [7. 0. 3. 0. 8. 0. 0. 0. 2.]
 [5. 0. 0. 0. 0. 7. 0. 3. 0.]
 [0. 5. 0. 0. 0. 1. 9. 7. 0.]
 [6. 0. 0. 7. 0. 9. 0. 0. 8.]
 [0. 4. 7. 2. 0. 0. 0. 5. 0.]
 [0. 2. 0. 6. 0. 0. 0. 0. 9.]
 [8. 0. 0. 0. 9. 0. 3. 0. 5.]
 [3. 0. 0. 8. 2. 0. 0. 1. 0.]]
solutions
 [[4 8 9 5 3 2 7 6 1]
 [7 1 3 4 8 6 5 9 2]
 [5 6 2 9 1 7 8 3 4]
 [2 5 8 3 4 1 9 7 6]
 [6 3 1 7 5 9 2 4 8]
 [9 4 7 2 6 8 1 5 3]
 [1 2 5 6 7 3 4 8 9]
 [8 7 6 1 9 4 3 2 5]
 [3 9 4 8 2 5 6 1 7]]


In [0]:
class ConvNet(nn.Module):
  def __init__(self, c_in, c_out, filt=3, stride=1, padding=1):
    super(ConvNet, self).__init__()

    self.conv = nn.Conv2d(c_in, c_out, (filt, filt), stride=stride, padding=padding)
    self.bn = nn.BatchNorm2d(c_out)
  
  def forward(self, x):
    n, c, h, w = x.shape
    z = F.relu(self.bn(self.conv(x)))
    return z

# n1 - number of conv layers with kernel size (3, 3) - try to learn local
# n2 - number of conv layers with kernel size (9, 9) - try to learn global
# c_mid - number of kernels in conv layers
class SudokuNet(nn.Module):
  # !!! c_out should be 10 !!!
  def __init__(self, n, n2, c_mid, c_in=1, c_out=10):
    super(SudokuNet, self).__init__()
    self.conv1 = ConvNet(c_in, c_mid)

    self.seq = nn.Sequential()
    for i in range(n):
      self.seq.add_module(str(i), ConvNet(c_mid, c_mid, 3, 1, 1))

    self.seq2 = nn.Sequential()
    for i in range(n2):
      self.seq2.add_module(str(i), ConvNet(c_mid, c_mid, 9, 1, 4))


    self.convLast = nn.Conv2d(c_mid, c_out, (3,3), stride=1, padding=1)
    self.logsoftmax = nn.LogSoftmax(dim=1)

    
  def forward(self, x):
    z = self.conv1(x)
    z = self.seq(z)
    z = self.seq2(z)
    z = self.logsoftmax(self.convLast(z))
    return z


In [0]:
# get accuracy in range [start, end) with batch_size
# change start/end to reduce running time
# choose wisely, so that |train| : |test| = 9 : 1 (or 99 : 1)
def get_accuracy(model, batch_size, start, end):
    model.train(mode=False)

    total_samples = 0
    total_correct = 0

    for i in range(start, end, batch_size):
        x = quizes[i:i+batch_size]
        y = solutions[i:i+batch_size]
        x = x[:, np.newaxis]

        X, y = torch.from_numpy(x), torch.from_numpy(y)

        if torch.cuda.is_available():
            X = X.cuda()
            y = y.cuda()

        y_ = model(X)
        _, y_ = torch.max(y_, axis=1)
        
        total_samples += 81 * batch_size
        sum_ = torch.sum(torch.eq(y_, y).type(torch.FloatTensor))
        total_correct += sum_.item()
        print(f'batch: {i}, correct: [{int(sum_.item())}/{81*batch_size}]')

    return total_correct / total_samples

In [10]:
n = 10
n2 = 0
c_mid=100
model = SudokuNet(n=n, n2=n2, c_mid=c_mid)
if torch.cuda.is_available():
    print("cuda")
    model.cuda()
else:
    print("cpu")

cuda


In [11]:
# load model from Sudoku_train.ipynb
checkpoint = torch.load("/content/drive/My Drive/Colab Notebooks/model_" + str(n) + "_" + str(n2) + "_" + str(c_mid) + ".tar")
model.load_state_dict(checkpoint['model_state_dict'])


<All keys matched successfully>

In [12]:
accuracy = get_accuracy(model, 1, 0, 30)
print(f"Total accuracy: {accuracy} on 30 tests")

batch: 0, correct: [78/81]
batch: 1, correct: [77/81]
batch: 2, correct: [77/81]
batch: 3, correct: [72/81]
batch: 4, correct: [70/81]
batch: 5, correct: [76/81]
batch: 6, correct: [54/81]
batch: 7, correct: [60/81]
batch: 8, correct: [57/81]
batch: 9, correct: [60/81]
batch: 10, correct: [55/81]
batch: 11, correct: [48/81]
batch: 12, correct: [62/81]
batch: 13, correct: [60/81]
batch: 14, correct: [65/81]
batch: 15, correct: [51/81]
batch: 16, correct: [58/81]
batch: 17, correct: [54/81]
batch: 18, correct: [55/81]
batch: 19, correct: [60/81]
batch: 20, correct: [62/81]
batch: 21, correct: [58/81]
batch: 22, correct: [53/81]
batch: 23, correct: [51/81]
batch: 24, correct: [68/81]
batch: 25, correct: [70/81]
batch: 26, correct: [57/81]
batch: 27, correct: [60/81]
batch: 28, correct: [68/81]
batch: 29, correct: [72/81]
Total accuracy: 0.7687242798353909 on 30 tests
